In [43]:
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import glob
import SimpleITK as sitk
import re
import random
random.seed(10)

import os
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
dicom.config.convert_wrong_length_to_UN = True

In [44]:
master_data = pd.read_csv('F:/U-CAN-Lymfom_A/Selected_for_UCAN_project/Selected_exams_from_U-CAN-Lymfom_A_MasterDataset.csv')
print(master_data.shape)
master_data.head()

(1382, 33)


,directory,root_directory,source_directory,patient_info,patient_directory,PET-CT_info,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
0,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr134758508773-20191212\CT-20191212-112330-3.7-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm_JK_,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr134758508773-20191212\CT-20191212-112330-3.7-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm_JK_,ASPTCTX0001_npr134758508773-20191212,CT-20191212-112330-3.7-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm_JK_,1.2.840.113619.2.5.42235195.12863.1576135318.405,npr134758508773,74.0,'72.0','1.64',512,512,442,0.976562,0.976562,3.00,"(512, 512, 442)","(0.976562, 0.976562, 3.0)","[-237.304, -233.886, 14.434]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842664208.103.1576074304.305.5240.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20191212','PET FDG WB 1 h','WB Venfas 3 ax','3.7 WB_FDG CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery MI'
1,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr134758508773-20191212\PT-20191212-110507-3.7-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm_JK_,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr134758508773-20191212\PT-20191212-110507-3.7-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm_JK_,ASPTCTX0001_npr134758508773-20191212,PT-20191212-110507-3.7-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm_JK_,1.2.840.113619.2.405.3842664208.1576145387.106224,npr134758508773,74.0,'72.0','1.64',256,256,395,1.953125,1.953125,2.79,"(256, 256, 395)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -1082.9699707031]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.3842664208.103.1576074304.305.5240.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['ORIGINAL', 'PRIMARY']",'20191212','PET FDG WB 1 h','QCFX-S 400','3.7 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",'PT','GE MEDICAL SYSTEMS','Discovery MI'
2,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr190483255078-20210203\CT-20210203-104152-3.12-WB_FDG-VENFAS--THORAX-INANDAD-WB-Venfas-3-ax-3.000000mm_JK_,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr190483255078-20210203\CT-20210203-104152-3.12-WB_FDG-VENFAS--THORAX-INANDAD-WB-Venfas-3-ax-3.000000mm_JK_,ASPTCTX0001_npr190483255078-20210203,CT-20210203-104152-3.12-WB_FDG-VENFAS--THORAX-INANDAD-WB-Venfas-3-ax-3.000000mm_JK_,1.2.840.113619.2.5.91447937.12518.1612342663.440,npr190483255078,75.0,'88.0','1.78',512,512,354,0.771484,0.771484,3.00,"(512, 512, 354)","(0.771484, 0.771484, 3.0)","[-197.885, -202.9, 4.309]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842663696.582.1612334019.8.6305.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20210203','PET FDG WB 1 h','WB Venfas 3 ax','3.12 WB_FDG VENFAS + THORAX INANDAD',NaN,'CT','GE MEDICAL SYSTEMS','Discovery MI'
3,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr190483255078-20210203\PT-20210203-102515-3.12-WB_FDG-VENFAS--THORAX-INANDAD-QCFX-S-400-M.Free-2.8000mm_JK_,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr190483255078-20210203\PT-20210203-102515-3.12-WB_FDG-VENFAS--THORAX-INANDAD-QCFX-S-400-M.Free-2.8000mm_JK_,ASPTCTX0001_npr190483255078-20210203,PT-20210203-102515-3.12-WB_FDG-VENFAS--THORAX-INANDAD-QCFX-S-400-M.Free-2.8000mm_JK_,1.2.840.113619.2.405.3842663696.1612344628.347570,npr190483255078,75.0,'88.0','1.78',256,256,314,1.953125,1.953125,2.80,"(256, 256, 314)","(1.953125, 1.953125, 2.8)","[-249.02343750000, -249.02343750000, -873.625000000

In [45]:
patdir_lst = list(set(master_data['patient_directory'].to_list()))

In [46]:
sampledir = random.sample(patdir_lst,5)
print(len(sampledir))
print(sampledir)

5
['ASPTCTX0001_npr459721086692-20160108', 'ASPTCTX0001_npr696287093420-20180814', 'ASPTCTX0001_npr391123015354-20150611', 'ASPTCTX0001_npr462153436708-20220302', 'ASPTCTX0001_npr647553057134-20190124']


In [47]:
sample_df = master_data[master_data.patient_directory.isin(sampledir)].copy()
print(sample_df.shape)
sample_df[sample_df['modality'].str.contains('CT')].shape

(10, 33)


(5, 33)

In [48]:
sample_df['num_slices'].to_list()

[354, 314, 323, 247, 215, 247, 1337, 287, 1154, 250]

In [49]:
def resample_volume(volume_path, interpolator = sitk.sitkLinear, new_size= [256, 256, 256], new_spacing = [2, 2, 3]):
    volume = sitk.ReadImage(volume_path, sitk.sitkFloat32) # read and cast to float32
    #original_spacing = volume.GetSpacing()
    #original_size = volume.GetSize()
    #new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(original_size, original_spacing, new_spacing)]
    return 

In [50]:
r"""
Skipped_images:
F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm
"""

'\nSkipped_images:\nF:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\\ASPTCTX0001_npr252097509348-20180509\\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm\n'

In [51]:
new_size = [384, 384, 384]
new_spacing = [2, 2, 3]

for index, row in sample_df.iterrows():

    #vol_img = sitk.ReadImage(row['directory'])
    
    print(row['directory'])

    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(row['directory'])
    reader.SetFileNames(dicom_names)
    vol_img = reader.Execute()
        
    #res_img = resample_volume(vol_img, sitk.sitkLinear, new_size, new_spacing)
    
    res_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkLinear,
                         vol_img.GetOrigin(), new_spacing, vol_img.GetDirection(), 0,
                         vol_img.GetPixelID())

    sitk.WriteImage(image = res_img, fileName = r'F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\Resampled_Selected_SampleImages\\' + 
                    str(row['patient_id']) + '_' + str(row['PET-CT_info']) + '_.nii.gz')



F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr462153436708-20220302\CT-20220302-131720-3.8-WB-F18-FDG-VENFAS-WB-Venfas-3-ax-3.000000mm_JK
F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr462153436708-20220302\PT-20220302-130601-3.8-WB-F18-FDG-VENFAS-QCFX-S-400-M.Free-2.8000mm_JK
F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr696287093420-20180814\CT-20180814-142729-6.5-UAS-WB-FDG-3D-CT-THORAX-BUK-NATIV-WB-NAT-AX-3.000000mm_JK
F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr696287093420-20180814\PT-20180814-140334-UAS-WB-FDG-3D-CT-THORAX-BUK-NATIV-WB-FDG-AC-3.2700mm_JK
F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr391123015354-20150611\CT-20150611-113405-6.1-UAS-WB-FDG-3D-STANDARD-CT-RECON-3.750000mm
F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr391123015354-20150611\PT-20150611-113541-UAS-WB-FDG-3D-UA

In [52]:
reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(r'F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm')
reader.SetFileNames(dicom_names)
vol_img = reader.Execute()

res_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkLinear,
                        vol_img.GetOrigin(), new_spacing, vol_img.GetDirection(), 0,
                        vol_img.GetPixelID())

sitk.WriteImage(image = res_img, fileName = r'F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\Resampled_Selected_SampleImages\\distorted_sample3Dimg_.nii.gz')
